In [1]:
import multiprocessing
import time

def worker1(shared_dict):
    """Worker 1: Sets an attribute and checks for worker 2's attribute"""
    print("Worker 1: Setting _worker_1_flag")
    shared_dict._worker_1_flag = "Hello from worker 1"
    
    time.sleep(1)  # Give worker 2 time to set its attribute
    
    # Check if we can see worker 2's attribute
    if hasattr(shared_dict, '_worker_2_flag'):
        print(f"Worker 1 SUCCESS: Can see worker 2's attribute: {shared_dict._worker_2_flag}")
    else:
        print("Worker 1 FAILURE: Cannot see worker 2's attribute")

def worker2(shared_dict):
    """Worker 2: Sets an attribute and checks for worker 1's attribute"""
    time.sleep(0.5)  # Let worker 1 go first
    
    print("Worker 2: Setting _worker_2_flag")
    shared_dict._worker_2_flag = "Hello from worker 2"
    
    # Check if we can see worker 1's attribute
    if hasattr(shared_dict, '_worker_1_flag'):
        print(f"Worker 2 SUCCESS: Can see worker 1's attribute: {shared_dict._worker_1_flag}")
    else:
        print("Worker 2 FAILURE: Cannot see worker 1's attribute")

def test_manager_attributes():
    """Test whether Manager dict attributes are truly shared across processes"""
    print("=== Testing Manager Dict Attribute Sharing ===")
    
    manager = multiprocessing.Manager()
    shared_dict = manager.dict()
    
    # Start both workers
    p1 = multiprocessing.Process(target=worker1, args=(shared_dict,))
    p2 = multiprocessing.Process(target=worker2, args=(shared_dict,))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    
    print("\n=== Results from Main Process ===")
    
    # Check from main process what attributes exist
    print(f"Main: _worker_1_flag exists? {hasattr(shared_dict, '_worker_1_flag')}")
    print(f"Main: _worker_2_flag exists? {hasattr(shared_dict, '_worker_2_flag')}")
    
    if hasattr(shared_dict, '_worker_1_flag'):
        print(f"Main: _worker_1_flag value = '{shared_dict._worker_1_flag}'")
    
    if hasattr(shared_dict, '_worker_2_flag'):
        print(f"Main: _worker_2_flag value = '{shared_dict._worker_2_flag}'")
    
    # Test the specific case from your code
    print(f"\nMain: Testing getattr pattern from your code...")
    seeds_populated = getattr(shared_dict, '_seeds_populated', False)
    print(f"Main: getattr(shared_dict, '_seeds_populated', False) = {seeds_populated}")
    
    shared_dict._seeds_populated = True
    seeds_populated_after = getattr(shared_dict, '_seeds_populated', False)
    print(f"Main: After setting, getattr(...) = {seeds_populated_after}")

def test_concurrent_flag_setting():
    """Test the exact pattern from your DRL code"""
    print("\n=== Testing Your Exact Pattern ===")
    
    def worker_with_flag_check(shared_dict, worker_id):
        print(f"Worker {worker_id}: Checking _seeds_populated flag")
        
        # This is your exact pattern
        if getattr(shared_dict, '_seeds_populated', False):
            print(f"Worker {worker_id}: Seeds already populated, exiting")
            return
        
        print(f"Worker {worker_id}: Seeds not populated, will populate")
        time.sleep(0.5)  # Simulate work
        
        # Set the flag
        shared_dict._seeds_populated = True
        print(f"Worker {worker_id}: Set _seeds_populated = True")
    
    manager = multiprocessing.Manager()
    shared_dict = manager.dict()
    
    # Start 3 workers simultaneously
    processes = []
    for i in range(3):
        p = multiprocessing.Process(target=worker_with_flag_check, args=(shared_dict, i))
        processes.append(p)
        p.start()
    
    for p in processes:
        p.join()
    
    print(f"\nFinal state: _seeds_populated = {getattr(shared_dict, '_seeds_populated', False)}")

if __name__ == "__main__":
    test_manager_attributes()
    test_concurrent_flag_setting()

=== Testing Manager Dict Attribute Sharing ===
Worker 1: Setting _worker_1_flag
Worker 2: Setting _worker_2_flag
Worker 2 FAILURE: Cannot see worker 1's attribute
Worker 1 FAILURE: Cannot see worker 2's attribute

=== Results from Main Process ===
Main: _worker_1_flag exists? False
Main: _worker_2_flag exists? False

Main: Testing getattr pattern from your code...
Main: getattr(shared_dict, '_seeds_populated', False) = False
Main: After setting, getattr(...) = True

=== Testing Your Exact Pattern ===
Worker 0: Checking _seeds_populated flag
Worker 0: Seeds not populated, will populateWorker 1: Seeds not populated, will populate
Worker 1: Checking _seeds_populated flag

Worker 2: Checking _seeds_populated flag
Worker 2: Seeds not populated, will populate
Worker 0: Set _seeds_populated = True
Worker 1: Set _seeds_populated = True
Worker 2: Set _seeds_populated = True

Final state: _seeds_populated = False
